In [ ]:
import os
import sys
os.environ["PYSPARK_SUBMIT_ARGS"]='--conf spark.sql.catalogImplementation=in-memory pyspark-shell'
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

In [ ]:
spark

![big_picture](https://spark.apache.org/docs/latest/img/structured-streaming-stream-as-a-table.png)

![programming_model](https://spark.apache.org/docs/latest/img/structured-streaming-model.png)

![quick_example](https://spark.apache.org/docs/latest/img/structured-streaming-example-model.png)

## A simple example

In [ ]:
!hdfs dfs -ls /user/pavel.klemenkov/lectures/lecture05/dummy_files/

In [ ]:
!hdfs dfs -tail /user/pavel.klemenkov/lectures/lecture05/dummy_files/file01.csv

In [ ]:
!hdfs dfs -tail /user/pavel.klemenkov/lectures/lecture05/dummy_files/file02.csv

In [ ]:
!hdfs dfs -tail /user/pavel.klemenkov/lectures/lecture05/dummy_files/file03.csv

In [ ]:
from pyspark.sql.types import *

In [ ]:
schema = StructType(fields=[
    StructField("animal", StringType()),
    StructField("count", IntegerType())
])

In [ ]:
df = spark.read.csv("/user/pavel.klemenkov/lectures/lecture05/dummy_files/file01.csv", sep="\t", schema=schema)

In [ ]:
df.show()

### Create a streaming DataFrame.

In [ ]:
streaming_df = spark.readStream\
                    .schema(schema)\
                    .option("maxFilesPerTrigger", 1)\
                    .csv("/user/pavel.klemenkov/lectures/lecture05/dummy_files/", sep="\t")

In [ ]:
streaming_df

In [ ]:
streaming_df.isStreaming

### Define some transformations on this DataFrame

In [ ]:
computation = streaming_df.select("animal")

In [ ]:
computation

### Define the output stream. `start()` method starts streaming computation and returns a `StreamingQuery`. It does not block

In [ ]:
query = computation.writeStream\
                   .format("memory")\
                   .queryName("some_query")\
                   .outputMode("append")\
                   .trigger(processingTime="30 seconds")\
                   .start()

In [ ]:
query

In [ ]:
query.isActive

In [ ]:
import time

In [ ]:
while True:
    spark.sql("select * from some_query").show()
    time.sleep(5)

In [ ]:
query.stop()

### Check whether query is still running

In [ ]:
query.isActive

### All the information about batches is available via `recentProgess` attribute

In [ ]:
query.recentProgress

In [ ]:
query.stop()

### Output modes

The “Output” is defined as what gets written out to the external storage. The output can be defined in a different mode:

**Complete Mode** - The entire updated Result Table will be written to the external storage. It is up to the storage connector to decide how to handle writing of the entire table.

**Append Mode** - Only the new rows appended in the Result Table since the last trigger will be written to the external storage. This is applicable only on the queries where existing rows in the Result Table are not expected to change.

**Update Mode** - Only the rows that were updated in the Result Table since the last trigger will be written to the external storage (available since Spark 2.1.1). Note that this is different from the Complete Mode in that this mode only outputs the rows that have changed since the last trigger. If the query doesn’t contain aggregations, it will be equivalent to Append mode.

In [ ]:
streaming_df = spark.readStream\
                    .schema(schema)\
                    .option("maxFilesPerTrigger", 1)\
                    .csv("/user/pavel.klemenkov/lectures/lecture05/dummy_files/", sep="\t")

In [ ]:
computation = streaming_df.groupBy("animal").count()

In [ ]:
query = computation.writeStream\
                   .format("memory")\
                   .queryName("some_query")\
                   .outputMode("complete")\
                   .trigger(processingTime="30 seconds")\
                   .start()

In [ ]:
while True:
    spark.sql("select * from some_query").show()
    time.sleep(5)

In [ ]:
query.stop()
query.isActive

In [ ]:
%pylab inline

In [ ]:
plt.rcParams["figure.figsize"] = 15, 10

In [ ]:
import seaborn as sns
sns.set_style("whitegrid")

In [ ]:
!hdfs dfs -ls /user/pavel.klemenkov/lectures/lecture05/events|head

There are about 50 JSON files in the directory. Let's see what each JSON file contains.

In [ ]:
!hdfs dfs -tail /user/pavel.klemenkov/lectures/lecture05/events/file-0.json

Each line in the file contains JSON record with two fields - `time` and `action`. Let's try to analyze these files interactively.

## Batch/Interactive Processing
The usual first step in attempting to process the data is to interactively query the data. Let's define a static DataFrame on the files, and give it a table name.

In [ ]:
from pyspark.sql.types import *

inputPath = "/user/pavel.klemenkov/lectures/lecture05/events/"

# Since we know the data format already, let's define the schema to speed up processing (no need for Spark to infer schema)
jsonSchema = StructType(fields = [ 
    StructField("time", TimestampType(), True), 
    StructField("action", StringType(), True) 
])

# Static DataFrame representing data in the JSON files
staticInputDF = spark.read\
                     .schema(jsonSchema)\
                     .json(inputPath)

In [ ]:
staticInputDF.show(5)

Now we can compute the number of "open" and "close" actions with one hour windows. To do this, we will group by the `action` column and 1 hour windows over the `time` column.

In [ ]:
import pyspark.sql.functions as f

In [ ]:
staticCountsDF = staticInputDF\
                  .groupBy(staticInputDF.action, f.window(staticInputDF.time, "1 hour"))\
                  .count()
        
staticCountsDF.cache()

# Register the DataFrame as table 'static_counts'
staticCountsDF.createOrReplaceTempView("static_counts")

Now we can directly use SQL to query the table. For example, here are the total counts across all the hours.

In [ ]:
staticCountsDF.show(truncate=False)

In [ ]:
spark.sql("select action, sum(count) as total_count from static_counts group by action").show()

In [ ]:
import seaborn as sns

In [ ]:
pdf = spark.sql("select action, sum(count) as total_count from static_counts group by action").toPandas()

In [ ]:
sns.barplot(x="action", y="total_count", data=pdf)

How about a timeline of windowed counts?

In [ ]:
query = """select action, date_format(window.end, "MMM-dd HH:mm") as time, count 
           from static_counts order by time, action
        """

spark.sql(query).show()

In [ ]:
pdf = spark.sql(query).toPandas()

In [ ]:
f = sns.barplot(x="time", y="count", hue="action", data=pdf)
for item in f.get_xticklabels():
    item.set_rotation(45)

Note the two ends of the graph. The close actions are generated such that they are after the corresponding open actions, so there are more "opens" in the beginning and more "closes" in the end.

## Stream Processing 
Now that we have analyzed the data interactively, let's convert this to a streaming query that continuously updates as data comes. Since we just have a static set of files, we are going to emulate a stream from them by reading one file at a time, in the chronological order they were created. The query we have to write is pretty much the same as the interactive query above.

In [ ]:
import pyspark.sql.functions as f

In [ ]:
# Similar to definition of staticInputDF above, just using `readStream` instead of `read`
streamingInputDF = spark.readStream\
                        .schema(jsonSchema)\
                        .option("maxFilesPerTrigger", 1)\
                        .json(inputPath)

In [ ]:
# Same query as staticInputDF
streamingCountsDF = streamingInputDF\
                     .groupBy(streamingInputDF.action, f.window(streamingInputDF.time, "1 hour"))\
                     .count()

In [ ]:
# Is this DF actually a streaming DF?
streamingCountsDF.isStreaming

As you can see, `streamingCountsDF` is a streaming Dataframe (`streamingCountsDF.isStreaming` was `true`). You can start streaming computation, by defining the sink and starting it. 
In our case, we want to interactively query the counts (same queries as above), so we will set the complete set of 1 hour counts to be in a in-memory table (note that this for testing purpose only in Spark 2.0).

In [ ]:
streaming_query = streamingCountsDF.writeStream\
                                   .format("memory")\
                                   .queryName("counts")\
                                   .outputMode("complete")\
                                   .start()

`query` is a handle to the streaming query that is running in the background. This query is continuously picking up files and updating the windowed counts. 

Note the status of query in the above cell. The progress bar shows that the query is active. 
Furthermore, if you expand the `> counts` above, you will find the number of files they have already processed. 

Let's wait a bit for a few files to be processed and then interactively query the in-memory `counts` table.

In [ ]:
query = """
    select action, date_format(window.end, "MMM-dd HH:mm") as time, count from counts order by time, action
"""

In [ ]:
import time

In [ ]:
from IPython.display import clear_output

In [ ]:
while True:
    clear_output()
    spark.sql(query).show()
    time.sleep(3)

In [ ]:
streaming_query.isActive

In [ ]:
streaming_query.stop()

## Watermarks

![](https://spark.apache.org/docs/latest/img/structured-streaming-late-data.png)

1. Define watermark delays on both inputs such that the engine knows how delayed the input can be (similar to streaming aggregations)

2. Define a constraint on event-time across the two inputs such that the engine can figure out when old rows of one input is not going to be required (i.e. will not satisfy the time constraint) for matches with the other input. This constraint can be defined in one of the two ways.

 1. Time range join conditions (e.g. ...JOIN ON leftTime BETWEEN rightTime AND rightTime + INTERVAL 1 HOUR),

 2. Join on event-time windows (e.g. ...JOIN ON leftTimeWindow = rightTimeWindow).



## Stream-to-stream joins

In [ ]:
from pyspark.sql.functions import rand

In [ ]:
spark.conf.set("spark.sql.shuffle.partitions", "1")

In [ ]:
impressions = (
  spark
    .readStream.format("rate").option("rowsPerSecond", "5").option("numPartitions", "1").load()
    .selectExpr("value AS adId", "timestamp AS impressionTime")
)

In [ ]:
clicks = (
  spark
  .readStream.format("rate").option("rowsPerSecond", "5").option("numPartitions", "1").load()
  .where((rand() * 100).cast("integer") < 30)
  .selectExpr("(value - 50) AS adId ", "timestamp AS clickTime")
  .where("adId > 0")
)  

In [ ]:
q = impressions.writeStream\
               .format("memory")\
               .queryName("imps")\
               .start()

In [ ]:
q.isActive

In [ ]:
spark.sql("select * from imps order by impressionTime desc").show(truncate=False)

In [ ]:
q.stop()

In [ ]:
imps_clicks = impressions.join(clicks, on="adId", how="inner")

In [ ]:
imps_clicks_q = imps_clicks.writeStream\
                           .format("memory")\
                           .queryName("imps_clicks_q")\
                           .start()
imps_clicks_q.isActive

In [ ]:
spark.sql("select * from imps_clicks_q order by impressionTime desc").show(truncate=False)

In [ ]:
imps_clicks_q.stop()

In [ ]:
from pyspark.sql.functions import expr

# Define watermarks
impressionsWithWatermark = impressions \
  .selectExpr("adId AS impressionAdId", "impressionTime") \
  .withWatermark("impressionTime", "10 seconds ")

clicksWithWatermark = clicks \
  .selectExpr("adId AS clickAdId", "clickTime") \
  .withWatermark("clickTime", "20 seconds")

In [ ]:
imps_clicks_watermark_outer = impressionsWithWatermark.join(
                                    clicksWithWatermark,
                                    expr(""" 
                                          clickAdId = impressionAdId AND 
                                          clickTime >= impressionTime AND 
                                          clickTime <= impressionTime + interval 1 minutes    
                                         """
                                        ),
                                    "leftOuter"
)

In [ ]:
imps_clicks_watermark_outer_q = imps_clicks_watermark_outer.writeStream\
                                                           .format("memory")\
                                                           .queryName("imps_clicks_watermark_outer_q")\
                                                           .start()
imps_clicks_watermark_outer_q.isActive

In [ ]:
while True:
    clear_output()
    spark.sql("select * from imps_clicks_watermark_outer_q where clickAdId is null").show(truncate=False)
    time.sleep(5)

In [ ]:
imps_clicks_watermark_outer_q.stop()

## Streams under the hood

![](pics/apache-spark-20-a-deep-dive-into-structured-streaming-by-tathagata-das-29-1024.jpg)

![](pics/apache-spark-20-a-deep-dive-into-structured-streaming-by-tathagata-das-30-1024.jpg)

![](pics/apache-spark-20-a-deep-dive-into-structured-streaming-by-tathagata-das-31-1024.jpg)

![](pics/apache-spark-20-a-deep-dive-into-structured-streaming-by-tathagata-das-32-1024.jpg)

![](pics/apache-spark-20-a-deep-dive-into-structured-streaming-by-tathagata-das-33-1024.jpg)

![](pics/apache-spark-20-a-deep-dive-into-structured-streaming-by-tathagata-das-34-1024.jpg)

![](pics/apache-spark-20-a-deep-dive-into-structured-streaming-by-tathagata-das-35-1024.jpg)

![](pics/apache-spark-20-a-deep-dive-into-structured-streaming-by-tathagata-das-38-1024.jpg)

![](pics/apache-spark-20-a-deep-dive-into-structured-streaming-by-tathagata-das-39-1024.jpg)

![](pics/apache-spark-20-a-deep-dive-into-structured-streaming-by-tathagata-das-40-1024.jpg)

## Concurrent streaming computations

In [ ]:
stream = spark.readStream\
              .format("rate")\
              .option("rowsPerSecond", "5")\
              .option("numPartitions", "1").load()

In [ ]:
stream.isStreaming

In [ ]:
import pyspark.sql.functions as F

In [ ]:
#spark.sparkContext.setLocalProperty("spark.scheduler.pool", "pool1")
q1 = stream.select("timestamp")\
           .writeStream\
           .format("parquet")\
           .option("checkpointLocation", "/user/pavel.klemenkov/checkpoint1")\
           .outputMode("append")\
           .start("/user/pavel.klemenkov/q1.parquet")

In [ ]:
#spark.sparkContext.setLocalProperty("spark.scheduler.pool", "pool2")
q2 = stream.select("timestamp")\
           .writeStream\
           .format("parquet")\
           .option("checkpointLocation", "/user/pavel.klemenkov/checkpoint2")\
           .outputMode("append")\
           .start("/user/pavel.klemenkov/q2.parquet")

In [ ]:
q1.stop()

In [ ]:
q2.stop()

## The same could be achieved in Python using `concurrent.futures` module

### Unsupported Operations

+ Multiple streaming aggregations (i.e. a chain of aggregations on a streaming DF) are not yet supported on streaming Datasets.

+ Limit and take first N rows are not supported on streaming Datasets.

+ Distinct operations on streaming Datasets are not supported.

+ Sorting operations are supported on streaming Datasets only after an aggregation and in Complete Output Mode.

+ Few types of outer joins on streaming Datasets are not supported. See the support matrix in the Join Operations section for more details.

In [ ]:
spark.stop()